# Template

This is the template project. Its very first content must be a Markdown header (e.g. `# My title`) and should be short and close to the project name.

In [ ]:
import hvplot.pandas  # noqa\n"
import pandas as pd
import panel as pn


## Data loading

Loading some data.

In [ ]:
df = pd.read_csv('data/athlete_events.csv')
df.head(100)

## Analysis

Creating an interactive pipeline.

In [ ]:
df = df.dropna(subset=["Weight", "Height"])

In [ ]:
avg_height_by_sport = df.groupby('Sport')['Height'].mean()
avg_height_by_sport.sort_values(inplace=True)
avg_height_by_sport.head()

In [ ]:
bar = avg_height_by_sport.hvplot.barh(by='Sport', stacked=True, height=600)
txt = avg_height_by_sport.hvplot.labels("Sport", "Height", "{Height:.0f} cm", invert=True, text_align="left", text_baseline="middle", text_color="darkred")
bar_sel = bar.select(Sport=["Basketball", "Gymnastics"]).opts(fill_color="darkred")
(bar * bar_sel * txt.select(Sport=["Basketball", "Gymnastics"])).opts(title="Average Height of each Olympic Sport, a 28 cm difference")

In [ ]:
avg_basketball_height = df[df["Sport"] == "Basketball"].groupby("Year").mean()
avg_gymnastics_height = df[df["Sport"] == "Gymnastics"].groupby("Year").mean()
avg_height_diff = avg_basketball_height["Height"] - avg_gymnastics_height["Height"]
avg_height_diff.hvplot.line()


In [ ]:
df = df.sort_values("Year")
df.hvplot("Year", "Height", groupby=["Medal", "Sport"])


In [ ]:
df.hvplot("Year", "Weight", groupby=["Medal", "Sport"])


In [ ]:
df.hvplot("Year", "Weight", groupby=["Medal", "Event"])


In [ ]:
medalist_avg = df[df["Medal"].notna()].groupby(["Year", "Event"]).mean()
not_medalist_avg = df[df["Medal"].isna()].groupby(["Year", "Event"]).mean()

height_medalist_plot = medalist_avg.hvplot.line(x="Year", y="Height", groupby=["Event"], label='Medalist')
height_not_medalist_plot = not_medalist_avg.hvplot.line(x="Year", y="Height", groupby=["Event"], color="red", label='Non-Medalist')
height_plot = (height_medalist_plot * height_not_medalist_plot).opts(title="Height", legend_position="bottom_right", x_range=(1880, 2000))

weight_medalist_plot = medalist_avg.hvplot.line(x="Year", y="Weight", groupby=["Event"], label='Medalist')
weight_not_medalist_plot = not_medalist_avg.hvplot.line(x="Year", y="Weight", groupby=["Event"], color="red", label='Non-Medalist')
weight_plot = (weight_medalist_plot * weight_not_medalist_plot).opts(title="Weight", legend_position="bottom_right", x_range=(1880, 2000))


combined_plots.cols(1)